## Loading of the necessary modules

In [1]:
# [Modules]
# General Python
import sys
import os
from math import sqrt
import random

# Data structures
import numpy as np  # For creating, manipulating arrays, and array calculations
import pandas as pd # For easy accessible tabular data frames, which can exported to Excel

# Image Processing
from scipy import ndimage as ndi # Imaging

# Regression
from lmfit import minimize, Parameters, report_fit
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from skimage import data, color

# Basic image display and plotting tools and notebook settings
import matplotlib as mpl             # Basic plotting module
from matplotlib import pyplot as plt # Basic plotting tools
# Set matplotlib in notebook mode and screen resolutions
%matplotlib inline
# mpl.rcParams['figure.dpi'] = 100     # Set display resolution
# #mpl.rcParams['savefig.dpi'] = 150    # Set figure save resolution
# import seaborn as sns                # Uses mpl, but makes nicer plots
# sns.set(style="whitegrid")           # Set sns style

# For fancy interactive Plotly graphs
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tools
from plotly.tools import FigureFactory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot # For plotly offline mode
# Set plotly to notebook mode
init_notebook_mode(connected=True)
py.sign_in('bjornharink_stanford', 'VqISgcHFwPkY0z50PG47')
#py.sign_in('login_name', 'df8aSdf07asd8faAsd78')  # Plot.ly login and API key, if you have one.

# Bead analysis module
#import bead_analysis as ba
import mrbles

Copyright 2017 - The Encoded Beads Project - ThornLab@UCSF and FordyceLab@Stanford


## The main script

### Main config

In [2]:
# Channel settings
DECODE_CHANNELS = slice('l-435','l-780')  # Channel range for decoding
OBJECT_CHANNEL = 'Brightfield'  # Channel for bead finding

# Filtering settings
reference_std_factor_low = 2   # Reference filter xSD from mean low
reference_std_factor_high = 2  # Reference filter xSD from mean high
back_std_factor = 2            # Background filter xSD from mean
set_prob = 2.3                   # Filter based on log probability

### Setting up and loading the reference images
Reference image locations and labels:

In [3]:
# Windows
REF_FOLDER = r"Z:\Data\Setup 1\Bead Reference Files\20170406 - Reference files - KARA"
# MAC OS X
# REF_FOLDER = "/mnt/fordyce_lab/Data/Setup 1/Bead Reference Files/20170406 - Reference files - KARA/"
REF_FILES = {"Dy" : os.path.join(REF_FOLDER, "Dy_Solos_3_MMStack_Pos0.ome.tif"),
             "Sm" : os.path.join(REF_FOLDER, "Sm_solos_5_MMStack_Pos0.ome.tif"),
             "Tm" : os.path.join(REF_FOLDER, "Tm_solos_3_MMStack_Pos0.ome.tif"),
             "Eu" : os.path.join(REF_FOLDER, "Eu_solos_6_MMStack_Pos0.ome.tif")}

REF_BEAD_SIZE = 16  # Estimated bead diameter in pixels
REF_DARK_NOISE = 99  # Camera dark noise

# Setup ref images object
spec_object = mrbles.pipeline.ReferenceSpectra(files = REF_FILES, 
                                       object_channel = OBJECT_CHANNEL, 
                                       channels = DECODE_CHANNELS, 
                                       bead_size = REF_BEAD_SIZE, 
                                       dark_noise = REF_DARK_NOISE)
spec_object.crop_x = slice(312, 712)
spec_object.crop_y = slice(312, 712)

# Background image set file location
BACK_FOLDER =  r"Z:\Data\Setup 1\Huy\20170823 HQN106\20170823_HQN_CN_HQN106_RemovedHigh_1uM_1"
BACK_FILE = os.path.join(BACK_FOLDER, "20170823_HQN_CN_HQN106_RemovedHigh_1uM_1_MMStack_Pos0.ome.tif")
BACK_ROIx = slice(438, 652)
BACK_ROIy = slice(420, 622)

NameError: name 'Spectra' is not defined

Process reference images and add Background Spectrum

In [ ]:
# Process ref images
spec_object.get_spectra()

# Process back image
spec_object.set_back(BACK_FILE, DECODE_CHANNELS, BACK_ROIx, BACK_ROIy)

# Export Sepctra object
ref_data_object = spec_object.output

# Plot spectra
ref_data_object.plot()

### Setting up bead library files

In [ ]:
# Target file location
TARGET_FOLDER = r"Z:\General\CodeSets"
#TARGET_FILE = os.path.join(TARGET_FOLDER, "20160226_DySmTm_48Codes.csv")
TARGET_FILE = os.path.join(TARGET_FOLDER, "20150714_DySmTm_48Codes.csv")
target_load = np.genfromtxt(TARGET_FILE, delimiter=',')
target = target_load[:, 1:4]  # Target set to Dy, Sm, Tm (no CeTb) Order: [CeTb, Dy, Sm, Tm]

# Bead library information
SEQ_FOLDER = r""
SEQ_FILE = os.path.join(SEQ_FOLDER, "code2Sequence.xlsx")
sequences = pd.read_excel(SEQ_FILE)

### Set up bead finding, icp, and gmix objects
Except for BEAD_SIZE, these can all stay the default values. They are just expclicity displayed for reference. Technically they can be called like:
- `icp = ba.ICP()`
- `gmix = ba.Classify(target)`

In [ ]:
bead_objects = mrbles.FindBeadsImaging(bead_size=12, border_clear=True)
bead_objects.eccen_param = 0.65
bead_objects.thr_block = 15
bead_objects.thr_c = 11
bead_objects.mask_bkg_buffer = 2
bead_objects.mask_bkg_size = 11

# ICP settings
icp = mrbles.ICP(matrix_method='std', max_iter=100, tol=1e-4, outlier_pct=0.001, train=False)

# GMM settings
gmix = mrbles.Classify(target, tol=1e-5, min_covar=1e-7, sigma=1e-5, train=True)

### Setting up bead images

In [ ]:
# Channel(s) settings
ASSAY_CHANNELS = ['Cy5']  # Must be list!

# slice(Y1, Y2) and slice(X1, X2) Y and X are reversed in array since rows (Y) go first and columns go second (X)
# General Region or interest
CROPx = slice(24, 1000)
CROPy = slice(24, 1000)

# Corresponding Flat-Field image
FF_FOLDER = r"Z:\Data\Setup 1\[Flat-Field and Dark-Field Setup 1]"
FF_FILE = os.path.join(FF_FOLDER, "20170815-FF-Cy5-500ms-169im-MED-BIN2-TopDown.tif")
flat_field = mrbles.tff.TiffFile(FF_FILE).asarray()
flat_field = flat_field/flat_field.max()  # Normalize Flat-Field

# Setting bead image folder and image patterns. This will select all images following the pattern.
BEAD_IMAGE_FOLDER = r"Z:\Data\Setup 1\Huy"
# BEAD_IMAGE_PATTERNS = {"NIR-PKIVIT-Flank" : r"20170802_HQN_PKIVIT_NIR_Flank_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "NIR-PKIVIT-Core" : r"20170802_HQN_PKIVIT_NIR_Pos1to6_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "NIR-PVIVIT-Flank" : r"20170919_HQN_CN_PVIVIT_Flank_NIR_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "NIR-PVIVIT-Core" : r"20170919_HQN_CN_PVIVIT_Pos1to6_NIR_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "NIR-AKAP79-Flank" : r"20170802_HQN_AKAP79_NIR_Flank_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif", 
#                        "NIR-AKAP79-Core" : r"20170802_HQN_AKAP79_NIR_Pos1to6_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "NIR-NFATc2-Flank" : "20170802_HQN_NFATc2_NIR_Flank_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        #"NIR-NFATc2-Core" : r"20170802_HQN_NFATc2_NIR_Pos1to6_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "NIR-NFATc2-Core" : r"20170802_HQN_NFATc2_NIR_Pos1to6_250nM_10ms_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "NIR-PxI2" : r"20171107_HQN_PxI2_NIR_100p200ms_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif"}

# BEAD_IMAGE_PATTERNS = {"aHis-PVIVIT-Core" : r"20170815_HQN_PVIVIT_Pos1to6_AntiHis_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "aHis-PVIVIT-Flank" : r"20170815_HQN_PVIVIT_Flank_AntiHis_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "aHis-PKIVIT-Core" : r"20170629_HQN_CN_PKIVIT_Pos1to6_Ab_control_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "aHis-PKIVIT-Flank" : r"20170629_HQN_CN_PKIVIT_Flank_Ab_control_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "aHis-AKAP79-Core" : r"20170815_HQN_AKAP79_Pos1to6_AntiHis_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "aHis-AKAP79-Flank" : r"20170815_HQN_AKAP79_Flank_AntiHis_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "aHis-NFATc2-Core" : r"20170815_HQN_NFATc2_Pos1to6_AntiHis_250nMb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "aHis-NFATc2-Flank" : r"20170815_HQN_NFATc2_Flank_AntiHis_250nM_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                        "aHis-PxI2" : "20171107_HQN_PxI2_aHis_100p200ms_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif"}

BEAD_IMAGE_PATTERNS = {"NIR-PxI2" : r"20171107_HQN_PxI2_NIR_100p200ms_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
                       "aHis-PxI2" : "20171107_HQN_PxI2_aHis_100p200ms_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif"}

#NORM_IMAGE_FOLDER = r"Z:\Data\Setup 1\Huy"
# NORM_IMAGE_PATTERN = {"AKAP79-Flank" : r"20170629_HQN_CN_PKIVIT_Pos1to6_Biotinb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                       "AKAP79-Core" : r"20170629_HQN_CN_PKIVIT_Pos1to6_Biotinb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                       "NFATc2-Flank" : r"20170629_HQN_CN_PKIVIT_Pos1to6_Biotinb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                       "NFATc2-Core" : r"20170629_HQN_CN_PKIVIT_Pos1to6_Biotinb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                       "PKIVIT-Flank" : r"20170629_HQN_CN_PKIVIT_Pos1to6_Biotinb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                       "PKIVIT-Core" : r"20170629_HQN_CN_PKIVIT_Pos1to6_Biotinb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                       "PVIVIT-Flank" : r"20170629_HQN_CN_PKIVIT_Pos1to6_Biotinb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif",
#                       "PVIVIT-Core" : r"20170629_HQN_CN_PKIVIT_Pos1to6_Biotinb_([1-9]|[1-9][0-9])_MMStack_Pos0.ome.tif"}
NORM_IMAGE_PATTERN = {}
NORM_IMAGE_PATTERN["AKAP79-Flank"] = pd.read_csv(r'G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20170811 - AKAP79\20170905 - 20170811-AKAP79-Redo-Flank - NewFF-LargerCircleROI - per bead - Biotin.csv')
NORM_IMAGE_PATTERN["AKAP79-Core"] = pd.read_csv(r'G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20170811 - AKAP79\20170905 - 20170811-AKAP79-Redo-Pos1to6 - NewFF-LargerCircleROI - per bead - Biotin.csv')
NORM_IMAGE_PATTERN["NFATc2-Flank"] = pd.read_csv(r'G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20170814 - NFATc2\20170830 - 20170814-NFATc2-Flank - REDO-NewFF-LargerCircleROI - per bead - Biotin.csv')
NORM_IMAGE_PATTERN["NFATc2-Core"] = pd.read_csv(r'G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20170814 - NFATc2\20170830 - 20170814-NFATc2-Pos1to6 - REDO-Correct2uMImg-NewFF-LargerCircleROI - per bead - Biotin.csv')
NORM_IMAGE_PATTERN["PKIVIT-Flank"] = pd.read_csv(r'G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20170829 - PKIVIT Redo\20170905 - 20170829-PKIVIT-Redo-Flank - NewFF-LargerCircleROI - per bead - Biotin.csv')
NORM_IMAGE_PATTERN["PKIVIT-Core"] = pd.read_csv(r'G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20170829 - PKIVIT Redo\20170905 - 20170829-PKIVIT-Redo-Pos1to6 - NewFF-LargerCircleROI - per bead - Biotin.csv')
NORM_IMAGE_PATTERN["PVIVIT-Flank"] = pd.read_csv(r'G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20170810 - PVIVIT\20170906 - 20170810-PVIVIT-Redo-Flank - NewFF-LargerCircleROI - per bead - Biotin.csv')
NORM_IMAGE_PATTERN["PVIVIT-Core"] = pd.read_csv(r'G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20170810 - PVIVIT\20170906 - 20170810-PVIVIT-Redo-Pos1to6 - NewFF-LargerCircleROI - per bead - Biotin.csv')
NORM_IMAGE_PATTERN["PxI2"] = pd.read_csv(r"PxIxIT2_biotin.csv")
NORM_CHANNEL = 'Cy5_5%'

# Search for files matching the patter in the bead image folder
#bead_image_files = ba.ImageSetRead.scan_path(BEAD_IMAGE_FOLDER, BEAD_IMAGE_PATTERNS['aHis-PVIVIT-Flank'])
#bead_image_files = ba.ImageSetRead.scan_path(NORM_IMAGE_FOLDER, NORM_IMAGE_PATTERN['biotin'])
# Load images. This may take some time.
#bead_image_obj = ba.ImageSetRead(bead_image_files)
#print(bead_image_obj.c_names) # Print channel names

# Image shift correction for assay channel(s)
shift_x = 0
shift_y = 0
CROPx_shift = slice(CROPx.start+shift_x, CROPx.stop+shift_x)
CROPy_shift = slice(CROPy.start+shift_y, CROPy.stop+shift_y)

In [ ]:
#bead_image_files

In [ ]:
def bead_data_function(image_folder, image_patterns, assay_channels, norm_data=None, norm_channel=None):
    bead_sets_per_bead = {}
    bead_sets_per_code = {}
    bead_images_channels = {}
    bead_set_data_per_bead = {}
    bead_set_data_per_code = {}
    labels_all_bead = {}
    labels_all_inside = {}
    labels_all_annulus = {}
    labels_all_bkg = {}
    for pattern_key, pattern in image_patterns.items():
        print("Analyzing condition: %s" %(pattern_key))
        # Search for files matching the patter in the bead image folder
        bead_image_files = mrbles.ImageSetRead.scan_path(image_folder, pattern)
        # Load images. This may take some time.
        bead_image_obj = mrbles.ImageSetRead(bead_image_files)
        # Load decode channsls
        bead_images_decode = bead_image_obj[:,DECODE_CHANNELS, CROPy, CROPx]
        # Load object channel
        bead_images_object = bead_image_obj[:,OBJECT_CHANNEL, CROPy_shift, CROPx_shift]
        # Load assay channel(s)
        if type(assay_channels) is not list:
            assay_channels = [assay_channels]
        for channel_idx, channel in enumerate(assay_channels):
            print("Analyzing channel: %s" %(channel))
            # Assign images, crop, and FF-correct
            assay_images = bead_image_obj[:,channel, CROPy_shift, CROPx_shift] / flat_field[CROPy_shift, CROPx_shift]
            
            # Setup Pandas DataFrame
            bead_set = \
                pd.DataFrame(columns=['img', 'lbl',                           # Image and label number of bead
                                      'dim_x', 'dim_y', 'dim_r', 'dim_area',  # Bead dimensions
                                      'bkg', 'ref',                           # Background and Reference data
                                      'rat_dy', 'rat_sm', 'rat_tm',           # Ratio data
                                       channel])                              # Assay data
            
            # Find beads and create masks
            labels_bead = []
            labels_inside = []
            labels_annulus = []
            labels_bkg = []
            skipped = []
            kept = list(range(len(bead_images_object)))
            bead_no = 0
            for idx in range(len(bead_images_object)):
                if bead_no == 0:
                    bead_objects.find(bead_images_object[idx], circle_size=340)
                    range_rad = [bead_objects._mask_radius]
                else:
                    bead_objects.find(bead_images_object[idx], circle_size=340)
                if bead_objects.bead_num < 1:
                    skipped.append(idx)
                    kept.remove(idx)
                    print("Removed image %i" % (idx+1))
                    continue
                print("Found %i beads in image %i" %(bead_objects.bead_num, idx+1))
                #labels_inside.append(bead_objects.mask_inside)
                labels_inside.append(bead_objects.morph_mask_step(-1,bead_objects.mask_inside))
                labels_bead.append(bead_objects.mask_bead)
                labels_annulus.append(bead_objects.mask_ring)
                labels_bkg.append(bead_objects.mask_bkg)
                circles_dim = bead_objects.bead_dims_bead.as_matrix()
                for lbl_idx, lbl in enumerate(bead_objects.bead_labels):
                    bead_set.loc[bead_no,('img', 'lbl', 
                                          'dim_x', 'dim_y', 'dim_r', 'dim_area')] = [idx, lbl, 
                         circles_dim[lbl_idx, 1], circles_dim[lbl_idx, 2], circles_dim[lbl_idx, 3], circles_dim[lbl_idx, 4]]
                    bead_no += 1
                    
#                 image_cc = color.gray2rgb(bead_objects._masked_img)
#                 dims = np.array(np.round(bead_objects.bead_dims_bead.loc[:,('y_centroid','x_centroid','radius')].values), dtype=np.int)
#                 for center_y, center_x, radius in zip(dims[:,0], dims[:,1], dims[:,2]):
#                     line_y = slice(center_y-radius, center_y+radius)
#                     line_x = slice(center_x-radius, center_x+radius)
#                     image_cc[center_y, line_x] = (20, 20, 220)
#                     image_cc[line_y, center_x] = (20, 20, 220)
#                 plt.figure(dpi=100)
#                 plt.imshow(image_cc, cmap=plt.cm.gray)
                
            print("Total number of beads found: %s" % bead_no)
            
            # Save labeled masks
            labels_all_bead[channel] = labels_bead
            labels_all_inside[channel] = labels_inside
            labels_all_annulus[channel] = labels_annulus
            labels_all_bkg[channel] = labels_bkg

            # Delete skipped images
            bead_images_object = np.delete(bead_images_object, skipped, axis=0)
            bead_images_decode = np.delete(bead_images_decode, skipped, axis=0)
            assay_images = np.delete(assay_images, skipped, axis=0)
            
            # Spectral unmix object
            spec_unmix = mrbles.SpectralUnmixing(ref_data_object)

            # Set bead number to 0
            bead_no = 0
            for lbls_idx, lbls in enumerate(labels_inside):
                # Get bead labels
                idx = bead_objects.get_unique_values(lbls)

                # Spectrally unmix
                spec_unmix.unmix(bead_images_decode[lbls_idx])

                # Set background and reference images
                background = spec_unmix['Bkg']  # Device background
                reference = spec_unmix['Eu']    # Internal reference: Eu

                # Make ratio images
                ratio_Dy = spec_unmix['Dy'] / reference
                ratio_Sm = spec_unmix['Sm'] / reference
                ratio_Tm = spec_unmix['Tm'] / reference

                # Get ratios from images
                ratio_data = np.empty((len(idx), target[0].size))
                ratio_data[:, 0] = ndi.labeled_comprehension(ratio_Dy, lbls, idx, np.median, float, -1)
                ratio_data[:, 1] = ndi.labeled_comprehension(ratio_Sm, lbls, idx, np.median, float, -1)
                ratio_data[:, 2] = ndi.labeled_comprehension(ratio_Tm, lbls, idx, np.median, float, -1)

                # Get background and reference data from ratio images
                background_data = ndi.labeled_comprehension(background, lbls, idx, np.median, float, -1)
                reference_data = ndi.labeled_comprehension(reference, lbls, idx, np.median, float, -1)

                # Get assay channel data
                assay_data = ndi.labeled_comprehension(assay_images[lbls_idx], labels_annulus[lbls_idx], idx, np.median, float, -1)

                # Subtract background global per image
                #assay_back_glob = ndi.labeled_comprehension(assay_images[lbls_idx], labels_bead[lbls_idx], 0, np.median, float, -1)
                #assay_data_glob = assay_data - assay_back_glob   # Subtract global background value
                #assay_data_glob[assay_data_glob < 0] = 0     # Set threshold to 0
                #assay_data_final = assay_data_glob    

                # Subtract background local per bead
                assay_back_local = ndi.labeled_comprehension(assay_images[lbls_idx], labels_bkg[lbls_idx], idx, np.median, float, -1)
                assay_data_local = assay_data - assay_back_local  # Subtract global background value
                #assay_data_local[assay_data_local < 0] = 0    # Set threshold to 0
                assay_data_final = assay_data_local

                for lbl in range(bead_objects.get_unique_count(lbls)):
                    bead_set.loc[bead_no,('rat_dy', 'rat_sm', 'rat_tm', 
                                          'bkg', 'ref', 
                                           channel)] = [ratio_data[lbl,0], ratio_data[lbl,1], ratio_data[lbl,2], 
                                                      background_data[lbl], reference_data[lbl], 
                                                      assay_data_final[lbl]]
                    bead_no += 1
                    
            # Filtering
            mask_bkg    = ( (bead_set.bkg > (bead_set.bkg.mean() - back_std_factor * bead_set.bkg.std())) &\
                            (bead_set.bkg < (bead_set.bkg.mean() + back_std_factor * bead_set.bkg.std())) )
            mask_ref    = ( (bead_set.ref > (bead_set.ref.mean() - reference_std_factor_low * bead_set.ref.std())) &\
                            (bead_set.ref < (bead_set.ref.mean() + reference_std_factor_high * bead_set.ref.std())) )
            filter_all = (mask_bkg & mask_ref)

            print("Pre filter: %s" % bead_set.index.size)
            print("Post filter: %s" % bead_set[filter_all].index.size)

            # Pre-filter post-filter images
            for x in range(3):
                idx = random.choice(range(len(assay_images)))
                fig = plt.figure(dpi=150)
                dims_post = bead_set.loc[(bead_set['img'] == kept[idx]), ('dim_x','dim_y','dim_r')].values
                ax = fig.add_subplot(1,2,1)
                ax.axis('off')
                fig.suptitle("Overlay Image Pre-filter image #: %s" % idx)
                mrbles.FindBeadsImaging.show_cross_overlay(image=bead_images_object[idx], dims=dims_post)
                #mrbles.FindBeadsImaging.show_image_overlay(assay_images[idx], labels_annulus[idx])
                #ax = fig.add_subplot(1,3,2)
                
                c_dims_post = bead_set.loc[filter_all & (bead_set['img'] == kept[idx]), ('dim_x','dim_y','dim_r')].values
                #ax.imshow(mrbles.FindBeadsImaging.cirle_overlay(assay_images[idx], dims=c_dims_post), 
                #          cmap='Greys_r')
                ax = fig.add_subplot(1,2,2)
                ax.axis('off')
                fig.suptitle("Overlay Image Post-filter image #: %s" % idx)
                mrbles.FindBeadsImaging.show_cross_overlay(image=bead_images_object[idx], dims=c_dims_post)
                
            # ICP
            icp.fit(bead_set.loc[filter_all, ('rat_dy', 'rat_sm', 'rat_tm')], target)
            bead_set = bead_set.combine_first(icp.transform())
            print("Tranformation matrix: ", icp.matrix)
            print("Offset vector: ", icp.offset)
            
            # GMM
            gmix.decode(bead_set.loc[filter_all, ('rat_dy_icp', 'rat_sm_icp', 'rat_tm_icp')])
            bead_set = bead_set.combine_first(gmix.output)
            print("Number of unique codes found:", gmix.found)
            print("Missing codes:", gmix.missing)
            s_score = silhouette_score(bead_set.loc[filter_all, ('rat_dy_icp', 'rat_sm_icp', 'rat_tm_icp')], gmix.output.code)
            print("Silhouette Coefficient:", s_score)
            print("AIC:", gmix._gmix.aic(bead_set.loc[filter_all, ('rat_dy_icp', 'rat_sm_icp', 'rat_tm_icp')]))
            print("BIC:", gmix._gmix.bic(bead_set.loc[filter_all, ('rat_dy_icp', 'rat_sm_icp', 'rat_tm_icp')]))
            
            print("Pre-filter:", bead_no)
            print("Pre log-prob filter:", bead_set.loc[(bead_set.code >= 0)].shape[0])
            print("Final beads no.:", bead_set.loc[(bead_set.code >= 0) & (bead_set.log_prob > set_prob)].shape[0])
            print("Total filtered out: %0.2f%%" % ((1-bead_set.loc[(bead_set.code >= 0) & (bead_set.log_prob > set_prob)].shape[0] / bead_set.shape[0])*100))
            print("Filtered/Total: %i/%i" % (bead_set.shape[0] - bead_set.loc[(bead_set.code >= 0) & (bead_set.log_prob > set_prob)].shape[0], bead_set.shape[0])) 
            
            bead_set_data_per_bead[channel] = bead_set
            
            biotin_string = pattern_key.split("-",1)[1]
            print(biotin_string)
            norm_data_sub = norm_data[biotin_string]
            
            #bead_set_data_per_code[channel] = mrbles.simp.get_stats_per_code(bead_set[bead_set.log_prob > set_prob], channel, norm=norm_data_sub[norm_data_sub.log_prob > set_prob], norm_channel=norm_channel, codes=48)
            bead_set_data_per_code[channel] = mrbles.simp.get_stats_per_code(bead_set[bead_set.log_prob > set_prob], channel, norm=norm_data_sub, norm_channel=norm_channel, codes=48)
            bead_images_channels[channel] = assay_images
        bead_sets_per_bead[pattern_key] = pd.concat(bead_set_data_per_bead)
        bead_sets_per_code[pattern_key] = pd.concat(bead_set_data_per_code)
    return pd.concat(bead_sets_per_bead), pd.concat(bead_sets_per_code)

#### Loading bead images, selecting bead images, and flatfield correction

In [ ]:
#bead_norm_per_bead

In [ ]:
#bead_norm_per_bead, bead_norm_per_code = bead_data_function(NORM_IMAGE_FOLDER, NORM_IMAGE_PATTERN, NORM_CHANNEL)

In [ ]:
#fig = ba.inspect.cluster3d_check(bead_norm_per_bead, target, gmix, set_prob=set_prob)
#py.iplot(fig, filename='TEST/3D-Cluster')  # Online

In [ ]:
bead_sets_per_bead, bead_sets_per_code = bead_data_function(BEAD_IMAGE_FOLDER, BEAD_IMAGE_PATTERNS, ASSAY_CHANNELS, NORM_IMAGE_PATTERN, NORM_CHANNEL)

### Per code stats

In [ ]:
bead_sets_per_code

### Per bead data

In [ ]:
bead_sets_per_bead

### Data export

In [ ]:
#folder_prefix = r"Z:\Data\Setup 1\Huy\20170731 PKIVIT\20170814 - Folder 20170731 PKIVIT - Pos1to6 - "
#folder_prefix = r"G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20171105 - NIR aHis\20171105 NIR - "
folder_prefix = r"G:\Team Drives\Calcineurin\CalcineurinBinding\Pyhton Data Analysis\20171105 - NIR aHis\20171109 PxI2 aHis-NIR - "

# Export to Excel.
# bead_sets_per_bead.to_excel(r"%sper bead - NOT NORM.xlsx"%folder_prefix)
# bead_sets_per_code.to_excel(r"%sper code - NORM.xlsx"%folder_prefix)
# bead_norm_per_bead.to_excel(r"%sper bead - Biotin.xlsx"%folder_prefix)
# bead_norm_per_code.to_excel(r"%sper code - Biotin.xlsx"%folder_prefix)

# Export to CSV.
#bead_sets_per_bead.to_csv(r"%sper bead - NOT NORM.csv"%folder_prefix)
#bead_sets_per_code.to_csv(r"%sper code - NORM.csv"%folder_prefix)
#bead_norm_per_bead.to_csv(r"%sper bead - Biotin.csv"%folder_prefix)
#bead_norm_per_code.to_csv(r"%sper code - Biotin.csv"%folder_prefix)

# Export to Python dataframe: Pickle. Best for keeping oroginal data structure.
# bead_sets_per_bead.to_pickle(r"%sper bead - NOT NORM.pickle"%folder_prefix)
# bead_sets_per_code.to_pickle(r"%sper code - NORM.pickle"%folder_prefix)
# bead_norm_per_bead.to_pickle(r"%sper bead - Biotin.pickle"%folder_prefix)
# bead_norm_per_code.to_pickle(r"%sper code - Biotin.pickle"%folder_prefix)

## Graphs

### Histogram bead data

In [ ]:
data = bead_sets_per_bead.loc[('2000 nM', 'Cy5')]
x1 = data.loc[(data.log_prob > set_prob)].Cy5

trace_1 = go.Histogram(
    name='2000 nM NFATc Pos1to6',
    x=x1,
    opacity=0.75
)

data = [trace_1]
layout = go.Layout(
    barmode='overlay',
    yaxis=dict(
        title="Fluorescence Intensity [A.U.]"
        )
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='TEST/Histogram')

### Bar-Plot - Separated Codes

In [ ]:
seq_names = []
seq_N = []
for seq_idx, seq_name in enumerate(sequences.sequence):
    seq_names.append("%i - %s" %(seq_idx+1, seq_name))

conc_name = '63 nM'
data_1 = bead_sets_per_code.loc[(conc_name, 'Cy5')]
N_1 = data_1.N.sum()
x1 = data_1.loc[(data_1.code >= 0)]

trace_1_bar = go.Bar(
    x=seq_names,
    y=x1.AVG_NORM,
    name="%s N=[%i]" %(conc_name, N_1),
    text=x1.N,
    error_y=dict(
        type='data',
        array=x1.SEM_NORM,
        visible=True
    )
)

conc_name = '2000 nM'
data_2 = bead_sets_per_code.loc[(conc_name, 'Cy5')]
N_2 = data_2.N.sum()
x2 = data_2.loc[(data_1.code >= 0)]

trace_2_bar = go.Bar(
    x=seq_names,
    y=x2.AVG_NORM,
    name="%s N=[%i]" %(conc_name, N_2),
    text=x2.N,
    error_y=dict(
        type='data',
        array=x2.SEM_NORM,
        visible=True
    )
)

data = [trace_1_bar, trace_2_bar]

layout = go.Layout( 
    yaxis=dict(
        title="Fluorescence Intensity [A.U.]"
        ),
    margin=go.Margin(
        b=140
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.2
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='CN/NFATc/Bar-Plot-Separate')

In [ ]:
concs = [63,125,250,500,1000,2000]
line_data = []
for seq_idx, seq in enumerate(sequences.sequence):
    conc_data = []
    conc_data_se = []
    for conc_label in concs:
        data = bead_sets_per_code[bead_sets_per_code.code==seq_idx].loc[('%s nM'%conc_label)]
        conc_data.append(data.MED_NORM) # MED_NORM for normalized Median, AVG_NORM for normalized Mean
        conc_data_se.append(data.SEM_NORM) # SEM_NORM for normalized SEM, SD_NORM for normalized SD
    line_data.append(
        go.Scatter(
            x=concs,
            y=conc_data,
            name=seq,
            error_y=dict(
                type='data',
                array=conc_data_se,
                visible=True
            )
        )
    )

layout = go.Layout( 
    title="Calcineurin - NFATc - Position 1-6 Sequences",
    xaxis=dict(
        title="Concentration [nM]"
        ),
    yaxis=dict(
        title="Mean Fluorescence Intensity +/- SD [A.U.]"
        ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    )
)

fig = go.Figure(data=line_data, layout=layout)
py.iplot(fig, filename='CN/NFATc/Scatter-Plot-All')